<a href="https://colab.research.google.com/github/elangbijak4/NLP-AI-Algorithm/blob/main/Kombinasi_Norvig_dan_N_Gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import re
from collections import Counter, defaultdict

In [9]:
# Contoh korpus sederhana dalam bahasa Indonesia
korpus = """
anak api buku bawa cantik dan dari dia dengan gajah gunung hutan ibu ingin itu jalan
jauh karena kerja kembali laut makan minum naik panas rumah sangat suka sudah tidak
"""

# Fungsi untuk membaca korpus kata dari string teks bahasa Indonesia
def words(text):
    return re.findall(r'\w+', text.lower())

# Fungsi untuk membuat model probabilitas dari korpus kata
def train(words):
    model = Counter(words)
    return model

# Membuat model probabilitas dari korpus kata
NWORDS = train(words(korpus))

# Karakter alfabet untuk bahasa Indonesia
alphabet = 'abcdefghijklmnopqrstuvwxyz'

# Fungsi untuk menghasilkan semua kemungkinan kata yang dihasilkan dari satu operasi pengeditan
def edits1(word):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in alphabet]
    inserts = [L + c + R for L, R in splits for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

# Fungsi untuk menghasilkan semua kemungkinan kata yang dihasilkan dari dua operasi pengeditan
def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

# Fungsi untuk menghasilkan semua kata yang diketahui (ada di dalam kamus)
def known(words):
    return set(w for w in words if w in NWORDS)

# Fungsi untuk memperbaiki kata yang salah eja menggunakan algoritma Norvig
def correct_with_norvig(word):
    candidates = (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
    return max(candidates, key=NWORDS.get)

# Fungsi untuk membangun model bigram dari korpus
def build_bigram_model(corpus):
    words = corpus.split()
    bigrams = [(words[i], words[i+1]) for i in range(len(words)-1)]
    model = defaultdict(Counter)
    for w1, w2 in bigrams:
        model[w1][w2] += 1
    return model

# Membangun model bigram
bigram_model = build_bigram_model(korpus)

# Fungsi untuk memperbaiki ejaan berdasarkan model bigram
def correct_with_bigram(word, model):
    if word in model:
        return max(model[word], key=model[word].get)
    else:
        return word  # Jika kata tidak ditemukan, kembalikan kata asli

# Fungsi untuk memperbaiki ejaan dengan menggunakan kombinasi Norvig dan N-Gram
def correct_with_norvig_and_bigram(word):
    norvig_correction = correct_with_norvig(word)
    bigram_correction = correct_with_bigram(norvig_correction, bigram_model)
    return bigram_correction

# Contoh penggunaan
print(correct_with_norvig_and_bigram("anak"))
print(correct_with_norvig_and_bigram("ape"))
print(correct_with_norvig_and_bigram("bukuu"))
print(correct_with_norvig_and_bigram("cantik"))

api
buku
bawa
dan
